In [1]:
import matplotlib
matplotlib.use("Agg")

from pyimagesearch.minigooglenet import MiniGoogLeNet
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.utils import multi_gpu_model
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import argparse

Using TensorFlow backend.


In [2]:
num_gpus = 2
output = 'multi_gpus.png'

In [3]:
G = num_gpus
OUTPUT = output

In [4]:
NUM_EPOCHS = 70
INIT_LR = 5e-3
 
def poly_decay(epoch):
    # initialize the maximum number of epochs, base learning rate,
    # and power of the polynomial
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0
 
    # compute the new learning rate based on polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
 
    # return the new learning rate
    return alpha

In [5]:
# load the training and testing data, converting the images from
# integers to floats
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")

In [6]:
# apply mean subtraction to the data
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean

In [7]:
# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [8]:
# construct the image generator for data augmentation and construct
# the set of callbacks
aug = ImageDataGenerator(width_shift_range=0.1,
    height_shift_range=0.1, horizontal_flip=True,
    fill_mode="nearest")
callbacks = [LearningRateScheduler(poly_decay)]

In [9]:
# check to see if we are compiling using just a single GPU
if G <= 1:
	print("[INFO] training with 1 GPU...")
	model = MiniGoogLeNet.build(width=32, height=32, depth=3,
		classes=10)
# otherwise, we are compiling using multiple GPUs
else:
	print("[INFO] training with {} GPUs...".format(G))
 
	# we'll store a copy of the model on *every* GPU and then combine
	# the results from the gradient updates on the CPU
	with tf.device("/device:GPU:0"):
		# initialize the model
		model = MiniGoogLeNet.build(width=32, height=32, depth=3,
			classes=10)
	
	# make the model parallel
	#model = multi_gpu_model(model, gpus=G)

[INFO] training with 2 GPUs...


In [10]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# train the network
print("[INFO] training network...")
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=64 * G),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // (64 * G),
	epochs=NUM_EPOCHS,
	callbacks=callbacks, verbose=2)

[INFO] compiling model...
[INFO] training network...
Epoch 1/70
 - 24s - loss: 1.5805 - acc: 0.4201 - val_loss: 1.2858 - val_acc: 0.5412
Epoch 2/70
 - 21s - loss: 1.1781 - acc: 0.5754 - val_loss: 1.4924 - val_acc: 0.4867
Epoch 3/70
 - 21s - loss: 1.0096 - acc: 0.6414 - val_loss: 1.1473 - val_acc: 0.5984
Epoch 4/70
 - 21s - loss: 0.8920 - acc: 0.6832 - val_loss: 1.0776 - val_acc: 0.6385
Epoch 5/70
 - 21s - loss: 0.8088 - acc: 0.7151 - val_loss: 0.7296 - val_acc: 0.7434
Epoch 6/70
 - 21s - loss: 0.7398 - acc: 0.7420 - val_loss: 0.8101 - val_acc: 0.7203
Epoch 7/70
 - 21s - loss: 0.6833 - acc: 0.7629 - val_loss: 0.8309 - val_acc: 0.7183
Epoch 8/70
 - 21s - loss: 0.6379 - acc: 0.7813 - val_loss: 0.7337 - val_acc: 0.7476
Epoch 9/70
 - 21s - loss: 0.5981 - acc: 0.7934 - val_loss: 0.6025 - val_acc: 0.7915
Epoch 10/70
 - 21s - loss: 0.5670 - acc: 0.8054 - val_loss: 0.7923 - val_acc: 0.7359
Epoch 11/70
 - 21s - loss: 0.5390 - acc: 0.8142 - val_loss: 0.7814 - val_acc: 0.7462
Epoch 12/70
 - 21s - 

In [15]:
# grab the history object dictionary
#H = H['history']
 
# plot the training loss and accuracy
N = np.arange(0, len(H["loss"]))
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H["loss"], label="train_loss")
plt.plot(N, H["val_loss"], label="test_loss")
plt.plot(N, H["acc"], label="train_acc")
plt.plot(N, H["val_acc"], label="test_acc")
plt.title("MiniGoogLeNet on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
 
# save the figure
plt.savefig(output)
plt.close()